In [4]:
#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

g = 1

frame = []
labdiff = []
sl = []
for k in range(1,12511):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  #image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  #image2 = np.float32(image2)
  #image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  #Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  #L2, a2, b2 = cv2.split(Lab2)
  blur = cv2.blur(L1,(5,5))
  m1 = np.mean(blur)
  sl.append(m1)
  frame.append(k)

#print(np.mean(L1))
print(sl)
print(frame)
np.savetxt('/content/sl.txt', sl, fmt='%s', delimiter=' ')
np.savetxt('/content/fr_sl.txt', frame, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION psi matrix fillup
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

frame = []
labdiff = []
sl = []
psi = []
for k in range(1,12510):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  image2 = np.float32(image2)
  image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  L2, a2, b2 = cv2.split(Lab2)
  blur1 = cv2.blur(L1,(5,5))
  m1 = np.mean(blur1)
  blur2 = cv2.blur(L2,(5,5))
  m2 = np.mean(blur2)
  frame.append(k) 
  if ((m1-m2) >= 0):
    psi.append(1)
  else:
    psi.append(-1)
  
#print(np.mean(L1))
print(psi)
print(frame)

print(psi[317])
print(psi[318])

np.savetxt('/content/psi.txt', frame, fmt='%s', delimiter=' ')

import numpy as np

sl = []
with open("/content/sl.txt", 'r') as fileobj:
    for row in fileobj:
      sl.append(row.rstrip('\n'))
        #print( row.rstrip('\n') )
print(sl)
sl = np.array(sl)
print(sl)

#FIRST PHASE OF GRADUAL TRANSITION 16/03/2021
import numpy as np

print(len(sl))
psi = []
frmpair = []
for k in range(0,12509):
  frmpair.append(k+1)
  d = np.float(sl[k]) - np.float(sl[k+1])
  if (d >= 0):
    psi.append(1)
  else:
    psi.append(-1)
#psi = np.array(psi)
np.savetxt('/content/psi.txt', psi, fmt='%s', delimiter=' ')
print(frmpair)
print(len(psi))
print(psi)

countp1 = 1
countm1 = 1
l = 0
t = 0

phi = np.zeros(((len(psi)),(len(psi))))
for k in range(0,(len(psi)-1)):
  if (psi[k] == -1) and (psi[k+1] == -1):
    #if (psi[k] == -1):
    countm1 = countm1 + 1
    print("no of -1")
    print(countm1)
    if (countm1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == 1):
    countp1 = countp1 + 1
    print("no of +1")
    print(countp1)
    if (countp1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == -1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
  if (psi[k] == -1) and (psi[k+1] == 1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
v1 = []
v2 = []
for c in range(0,len(phi)):
  if(phi[c][0]!=0):
    v1.append(phi[c][0])
    
    v2.append(phi[c][0] + np.count_nonzero(phi[c][:]) - 1)

print(phi)
print((v1))
print((v2))
v1 = np.array(v1)
v2 = np.array(v2)
#np.savetxt('/content/v1.txt', v1, fmt='%s', delimiter=' ')
v12 = np.stack((v1, v2), axis = 1)
np.savetxt('/content/v12.txt', v12, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#LAST VALIDATION OF GRADUAL CUT
st_fr = []
end_fr = []
for i in range(1,len(labdiff)):
  if(labdiff[i] > 0.03):
    st_fr.append(np.int(v1[i]))
    end_fr.append(np.int(v2[i]) )
print(st_fr)
print(end_fr)
len(st_fr)
st_fr = np.array(st_fr)
end_fr = np.array(end_fr)
featex = np.stack((st_fr, end_fr), axis = 1)
print(featex)
np.savetxt('/content/grad_cut_gt_pred.txt', featex, fmt='%s', delimiter=' ')
print(len(featex))

#EXTRACTING GROUNDTRUTH FROM XML FILE FOR ABRUPT CUT
import numpy as np
import xml.etree.ElementTree as ET
mytree = ET.parse('/content/ref_nad57.xml')
myroot = mytree.getroot()
print(int(myroot.attrib.get('totalFNum')))
totfno = int(myroot.attrib.get('totalFNum'))
prefno = []
postno = []
pref = []
postf = []
classi1 = []
cl = []
for x in myroot.findall('trans'):
  it1 = x.attrib.get('preFNum')

  it2 = x.attrib.get('postFNum')
  item =x.attrib.get('type')
  #print(item)
  prefno.append(int(it1))
  postno.append(int(it2))
  pref.append("frame%d.jpg" % int(it1))
  postf.append("frame%d.jpg" % int(it2))
  classi1.append(item)
  pre11 = []
  post11 = []
for i in range(len(classi1)):
  if (classi1[i]=='DIS' or classi1[i]=='FOI'):
    pre11.append(prefno[i])
    post11.append(postno[i])

#cl1 = np.array(cl)
pre1 = np.array(pre11)
post1 = np.array(post11)

feat1 = np.stack((pre1, post1), axis = 1)
print(feat1)
print(len(feat1))

np.savetxt('/content/gr_cut_gt.txt', feat1, fmt='%s', delimiter=' ')

#pre = 0
#RECALL CALCULATION
mat = np.zeros((len(feat1), len(featex)))
for i in range(0, len(featex)):
  a = featex[i][0]
  b = featex[i][1]
  for j in range(0, len(feat1)):
      r = range(feat1[j][0], feat1[j][1] + 1) 
      if((a in r) or (b in r)):
        mat[j][i] = 1

print(mat)
#print(mat[0][8])
print(len(mat))
 
print(mat.sum(axis = 1))
pre_mat = mat.sum(axis = 1)
print(pre_mat)
print(pre_mat.shape)
print(pre_mat[0])
print(np.sum(pre_mat))

pre = 0
for i in range(0, len(pre_mat)):
  a = pre_mat[i]
  #print(a)
  if(a!= 0.0):
    pre = pre + 1

print("recall")
print(pre/len(pre_mat))


Streaming output truncated to the last 5000 lines.
2
no of -1
3
no of -1
4
no of -1
5
no of -1
6
no of -1
7
no of -1
8
no of -1
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
1
1
1
1
no of -1
2
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
1
1
1
1
no of -1
2
1
1
1
1
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
no of +1
10
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
1
1
no of -1
2
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
1
1
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
no of -1
6
no of -1
7
no of -1
8
no of -1
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
no of -1
14
no of -1
15
no of -1
16
no of -1
17
no of -1
18
no of -1
19
no of -1
20
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
1
1
no of +1
2
no of +1
3
1
1
no of -1
2
no of -1
3

In [2]:
##DECOMPOSITION of video frames 14/03/2021
import sys
import shutil

import numpy as np
from math import floor
import pandas as pd
import numpy as np
import cv2

import os
import time
from PIL import Image
import math

# command line arguments --> file name, video_file_name, gpu or cpu 


# first decompose the video to frames
# place the video to be detected into the directory


#video = sys.argv[1]
#pred_text_file_name = sys.argv[2]

# resize image
#output = cv2.resize(src, dsize)


start = time.time()
text_file = 'frames.txt'

print('decomposing video to frames this may take a while  for large videos :) .....')
frames_path = 'video_frames/'
os.makedirs('video_frames/', exist_ok=True)
#os.makedirs('predictions/', exist_ok=True)

f = open(text_file, 'w+')
# Opens the Video file
cap= cv2.VideoCapture('/content/NAD57.mpg')
i=1
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    scale_percent = 25

#calculate the 50 percent of original dimensions
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)

# dsize
    dsize = (width, height)

    output = cv2.resize(frame, dsize) 
    frame_path = frames_path + 'frame_' + str(i) + '.jpg'
    cv2.imwrite("/content/video_frames/frame_"+ str(i) + ".jpg", output)
    f.write(frame_path + '\n')
    i+=1

cap.release()
cv2.destroyAllWindows()
print("decomposition complete")

end = time.time()
# total time taken
print(f"Runtime of the program is {end - start}")

decomposing video to frames this may take a while  for large videos :) .....
decomposition complete
Runtime of the program is 10.612298727035522


In [5]:
#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

g = 2

frame = []
labdiff = []
sl = []
for k in range(1,12511):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  #image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  #image2 = np.float32(image2)
  #image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  #Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  #L2, a2, b2 = cv2.split(Lab2)
  blur = cv2.blur(L1,(5,5))
  m1 = np.mean(blur)
  sl.append(m1)
  frame.append(k)

#print(np.mean(L1))
print(sl)
print(frame)
np.savetxt('/content/sl.txt', sl, fmt='%s', delimiter=' ')
np.savetxt('/content/fr_sl.txt', frame, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION psi matrix fillup
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

frame = []
labdiff = []
sl = []
psi = []
for k in range(1,12510):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  image2 = np.float32(image2)
  image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  L2, a2, b2 = cv2.split(Lab2)
  blur1 = cv2.blur(L1,(5,5))
  m1 = np.mean(blur1)
  blur2 = cv2.blur(L2,(5,5))
  m2 = np.mean(blur2)
  frame.append(k) 
  if ((m1-m2) >= 0):
    psi.append(1)
  else:
    psi.append(-1)
  
#print(np.mean(L1))
print(psi)
print(frame)

print(psi[317])
print(psi[318])

np.savetxt('/content/psi.txt', frame, fmt='%s', delimiter=' ')

import numpy as np

sl = []
with open("/content/sl.txt", 'r') as fileobj:
    for row in fileobj:
      sl.append(row.rstrip('\n'))
        #print( row.rstrip('\n') )
print(sl)
sl = np.array(sl)
print(sl)

#FIRST PHASE OF GRADUAL TRANSITION 16/03/2021
import numpy as np

print(len(sl))
psi = []
frmpair = []
for k in range(0,12509):
  frmpair.append(k+1)
  d = np.float(sl[k]) - np.float(sl[k+1])
  if (d >= 0):
    psi.append(1)
  else:
    psi.append(-1)
#psi = np.array(psi)
np.savetxt('/content/psi.txt', psi, fmt='%s', delimiter=' ')
print(frmpair)
print(len(psi))
print(psi)

countp1 = 1
countm1 = 1
l = 0
t = 0

phi = np.zeros(((len(psi)),(len(psi))))
for k in range(0,(len(psi)-1)):
  if (psi[k] == -1) and (psi[k+1] == -1):
    #if (psi[k] == -1):
    countm1 = countm1 + 1
    print("no of -1")
    print(countm1)
    if (countm1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == 1):
    countp1 = countp1 + 1
    print("no of +1")
    print(countp1)
    if (countp1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == -1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
  if (psi[k] == -1) and (psi[k+1] == 1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
v1 = []
v2 = []
for c in range(0,len(phi)):
  if(phi[c][0]!=0):
    v1.append(phi[c][0])
    
    v2.append(phi[c][0] + np.count_nonzero(phi[c][:]) - 1)

print(phi)
print((v1))
print((v2))
v1 = np.array(v1)
v2 = np.array(v2)
#np.savetxt('/content/v1.txt', v1, fmt='%s', delimiter=' ')
v12 = np.stack((v1, v2), axis = 1)
np.savetxt('/content/v12.txt', v12, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#LAST VALIDATION OF GRADUAL CUT
st_fr = []
end_fr = []
for i in range(1,len(labdiff)):
  if(labdiff[i] > 0.03):
    st_fr.append(np.int(v1[i]))
    end_fr.append(np.int(v2[i]) )
print(st_fr)
print(end_fr)
len(st_fr)
st_fr = np.array(st_fr)
end_fr = np.array(end_fr)
featex = np.stack((st_fr, end_fr), axis = 1)
print(featex)
np.savetxt('/content/grad_cut_gt_pred.txt', featex, fmt='%s', delimiter=' ')
print(len(featex))

#EXTRACTING GROUNDTRUTH FROM XML FILE FOR ABRUPT CUT
import numpy as np
import xml.etree.ElementTree as ET
mytree = ET.parse('/content/ref_nad57.xml')
myroot = mytree.getroot()
print(int(myroot.attrib.get('totalFNum')))
totfno = int(myroot.attrib.get('totalFNum'))
prefno = []
postno = []
pref = []
postf = []
classi1 = []
cl = []
for x in myroot.findall('trans'):
  it1 = x.attrib.get('preFNum')

  it2 = x.attrib.get('postFNum')
  item =x.attrib.get('type')
  #print(item)
  prefno.append(int(it1))
  postno.append(int(it2))
  pref.append("frame%d.jpg" % int(it1))
  postf.append("frame%d.jpg" % int(it2))
  classi1.append(item)
  pre11 = []
  post11 = []
for i in range(len(classi1)):
  if (classi1[i]=='DIS' or classi1[i]=='FOI'):
    pre11.append(prefno[i])
    post11.append(postno[i])

#cl1 = np.array(cl)
pre1 = np.array(pre11)
post1 = np.array(post11)

feat1 = np.stack((pre1, post1), axis = 1)
print(feat1)
print(len(feat1))

np.savetxt('/content/gr_cut_gt.txt', feat1, fmt='%s', delimiter=' ')

#pre = 0
#RECALL CALCULATION
mat = np.zeros((len(feat1), len(featex)))
for i in range(0, len(featex)):
  a = featex[i][0]
  b = featex[i][1]
  for j in range(0, len(feat1)):
      r = range(feat1[j][0], feat1[j][1] + 1) 
      if((a in r) or (b in r)):
        mat[j][i] = 1

print(mat)
#print(mat[0][8])
print(len(mat))
 
print(mat.sum(axis = 1))
pre_mat = mat.sum(axis = 1)
print(pre_mat)
print(pre_mat.shape)
print(pre_mat[0])
print(np.sum(pre_mat))

pre = 0
for i in range(0, len(pre_mat)):
  a = pre_mat[i]
  #print(a)
  if(a!= 0.0):
    pre = pre + 1

print("recall")
print(pre/len(pre_mat))


Streaming output truncated to the last 5000 lines.
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
1
1
1
1
no of -1
2
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
1
1
1
1
no of -1
2
1
1
1
1
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
no of +1
10
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
1
1
no of -1
2
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
1
1
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
no of -1
6
no of -1
7
no of -1
8
no of -1
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
no of -1
14
no of -1
15
no of -1
16
no of -1
17
no of -1
18
no of -1
19
no of -1
20
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
1
1
no of +1
2
no of +1
3
1
1
no of -1
2
no of -1
3
no of -1
4
1
1
no of +1
2
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
no

In [6]:
#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

g = 3

frame = []
labdiff = []
sl = []
for k in range(1,12511):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  #image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  #image2 = np.float32(image2)
  #image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  #Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  #L2, a2, b2 = cv2.split(Lab2)
  blur = cv2.blur(L1,(5,5))
  m1 = np.mean(blur)
  sl.append(m1)
  frame.append(k)

#print(np.mean(L1))
print(sl)
print(frame)
np.savetxt('/content/sl.txt', sl, fmt='%s', delimiter=' ')
np.savetxt('/content/fr_sl.txt', frame, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 15/03/2021 GRADUAL TRANSITION psi matrix fillup
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math

frame = []
labdiff = []
sl = []
psi = []
for k in range(1,12510):
  image1 = cv2.imread("/content/video_frames/frame_"+ str(k) + ".jpg")
  image2 = cv2.imread("/content/video_frames/frame_"+ str(k+1) + ".jpg")
  image1 = np.float32(image1)
  image1 *= 1./255
  image2 = np.float32(image2)
  image2 *= 1./255

  Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
  Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


  L1, a1, b1 = cv2.split(Lab1)
  L2, a2, b2 = cv2.split(Lab2)
  blur1 = cv2.blur(L1,(5,5))
  m1 = np.mean(blur1)
  blur2 = cv2.blur(L2,(5,5))
  m2 = np.mean(blur2)
  frame.append(k) 
  if ((m1-m2) >= 0):
    psi.append(1)
  else:
    psi.append(-1)
  
#print(np.mean(L1))
print(psi)
print(frame)

print(psi[317])
print(psi[318])

np.savetxt('/content/psi.txt', frame, fmt='%s', delimiter=' ')

import numpy as np

sl = []
with open("/content/sl.txt", 'r') as fileobj:
    for row in fileobj:
      sl.append(row.rstrip('\n'))
        #print( row.rstrip('\n') )
print(sl)
sl = np.array(sl)
print(sl)

#FIRST PHASE OF GRADUAL TRANSITION 16/03/2021
import numpy as np

print(len(sl))
psi = []
frmpair = []
for k in range(0,12509):
  frmpair.append(k+1)
  d = np.float(sl[k]) - np.float(sl[k+1])
  if (d >= 0):
    psi.append(1)
  else:
    psi.append(-1)
#psi = np.array(psi)
np.savetxt('/content/psi.txt', psi, fmt='%s', delimiter=' ')
print(frmpair)
print(len(psi))
print(psi)

countp1 = 1
countm1 = 1
l = 0
t = 0

phi = np.zeros(((len(psi)),(len(psi))))
for k in range(0,(len(psi)-1)):
  if (psi[k] == -1) and (psi[k+1] == -1):
    #if (psi[k] == -1):
    countm1 = countm1 + 1
    print("no of -1")
    print(countm1)
    if (countm1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == 1):
    countp1 = countp1 + 1
    print("no of +1")
    print(countp1)
    if (countp1 > g):
      phi[l][t] = ((k+2))
      t = t + 1
    else:
      l = l + 1
  if (psi[k] == 1) and (psi[k+1] == -1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
  if (psi[k] == -1) and (psi[k+1] == 1):
    countp1 = 1
    countm1 = 1
    t = 0
    print(countp1)
    print(countm1)
v1 = []
v2 = []
for c in range(0,len(phi)):
  if(phi[c][0]!=0):
    v1.append(phi[c][0])
    
    v2.append(phi[c][0] + np.count_nonzero(phi[c][:]) - 1)

print(phi)
print((v1))
print((v2))
v1 = np.array(v1)
v2 = np.array(v2)
#np.savetxt('/content/v1.txt', v1, fmt='%s', delimiter=' ')
v12 = np.stack((v1, v2), axis = 1)
np.savetxt('/content/v12.txt', v12, fmt='%s', delimiter=' ')

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#CIEDE2000 frame sequence difference 14/03/2021 ORIGINAL PAPER with change from first gradual transition
import cv2
import numpy as np
from skimage import color
#from ciede2000 import CIEDE2000
import math
import time
from matplotlib import pyplot as plt
import numpy as np

def CIEDE2000(Lab_1, Lab_2):
    '''Calculates CIEDE2000 color distance between two CIE L*a*b* colors'''
    C_25_7 = 6103515625 # 25**7
    
    L1, a1, b1 = Lab_1[0], Lab_1[1], Lab_1[2]
    L2, a2, b2 = Lab_2[0], Lab_2[1], Lab_2[2]
    C1 = math.sqrt(a1**2 + b1**2)
    C2 = math.sqrt(a2**2 + b2**2)
    C_ave = (C1 + C2) / 2
    G = 0.5 * (1 - math.sqrt(C_ave**7 / (C_ave**7 + C_25_7)))
    
    L1_, L2_ = L1, L2
    a1_, a2_ = (1 + G) * a1, (1 + G) * a2
    b1_, b2_ = b1, b2
    
    C1_ = math.sqrt(a1_**2 + b1_**2)
    C2_ = math.sqrt(a2_**2 + b2_**2)
    
    if b1_ == 0 and a1_ == 0: h1_ = 0
    elif a1_ >= 0: h1_ = math.atan2(b1_, a1_)
    else: h1_ = math.atan2(b1_, a1_) + 2 * math.pi
    
    if b2_ == 0 and a2_ == 0: h2_ = 0
    elif a2_ >= 0: h2_ = math.atan2(b2_, a2_)
    else: h2_ = math.atan2(b2_, a2_) + 2 * math.pi

    dL_ = L2_ - L1_
    dC_ = C2_ - C1_    
    dh_ = h2_ - h1_
    if C1_ * C2_ == 0: dh_ = 0
    elif dh_ > math.pi: dh_ -= 2 * math.pi
    elif dh_ < -math.pi: dh_ += 2 * math.pi        
    dH_ = 2 * math.sqrt(C1_ * C2_) * math.sin(dh_ / 2)
    
    L_ave = (L1_ + L2_) / 2
    C_ave = (C1_ + C2_) / 2
    
    _dh = abs(h1_ - h2_)
    _sh = h1_ + h2_
    C1C2 = C1_ * C2_
    
    if _dh <= math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2
    elif _dh  > math.pi and _sh < 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 + math.pi
    elif _dh  > math.pi and _sh >= 2 * math.pi and C1C2 != 0: h_ave = (h1_ + h2_) / 2 - math.pi 
    else: h_ave = h1_ + h2_
    
    T = 1 - 0.17 * math.cos(h_ave - math.pi / 6) + 0.24 * math.cos(2 * h_ave) + 0.32 * math.cos(3 * h_ave + math.pi / 30) - 0.2 * math.cos(4 * h_ave - 63 * math.pi / 180)
    
    h_ave_deg = h_ave * 180 / math.pi
    if h_ave_deg < 0: h_ave_deg += 360
    elif h_ave_deg > 360: h_ave_deg -= 360
    dTheta = 30 * math.exp(-(((h_ave_deg - 275) / 25)**2))
    
    R_C = 2 * math.sqrt(C_ave**7 / (C_ave**7 + C_25_7))  
    S_C = 1 + 0.045 * C_ave
    S_H = 1 + 0.015 * C_ave * T
    
    Lm50s = (L_ave - 50)**2
    S_L = 1 + 0.015 * Lm50s / math.sqrt(20 + Lm50s)
    R_T = -math.sin(dTheta * math.pi / 90) * R_C

    k_L, k_C, k_H = 1, 1, 1
    
    f_L = dL_ / k_L / S_L
    f_C = dC_ / k_C / S_C
    f_H = dH_ / k_H / S_H
    
    dE_00 = math.sqrt(f_L**2 + f_C**2 + f_H**2 + R_T * f_C * f_H)
    return dE_00
#start time
start = time.time()
frame = []
labdiff = []
for k in range(0,(len(v1) - 1)):
  if (((np.int(v1[k])) - 5) == 0 or ((np.int(v1[k])) - 5) < 0):
    #image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    #image1 = np.float32(image1)
    #image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    #Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    #L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab2.shape[0]):
      for j in range(0,Lab2.shape[1]):
        diff = diff + CIEDE2000((0, 0, 0), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

  else:
    image1 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v1[k])) - 5) + ".jpg")
    image2 = cv2.imread("/content/video_frames/frame_"+ str((np.int(v2[k])) +  5) + ".jpg")
    image1 = np.float32(image1)
    image1 *= 1./255
    image2 = np.float32(image2)
    image2 *= 1./255

    Lab1 = cv2.cvtColor(image1, cv2.COLOR_BGR2Lab)
  #print(Lab1.shape)
    Lab2 = cv2.cvtColor(image2, cv2.COLOR_BGR2Lab)


    L1, a1, b1 = cv2.split(Lab1)
    L2, a2, b2 = cv2.split(Lab2)

#print(np.mean(L1))
#print(a1)
    diff = 0
    for i in range(0,Lab1.shape[0]):
      for j in range(0,Lab1.shape[1]):
        diff = diff + CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j]))
    #print(CIEDE2000((L1[i][j], a1[i][j], b1[i][j]), (L2[i][j], a2[i][j], b2[i][j])))
    labdiff.append(diff/(60*88*3))

print(labdiff)
labdiff = np.array(labdiff)

#LAST VALIDATION OF GRADUAL CUT
st_fr = []
end_fr = []
for i in range(1,len(labdiff)):
  if(labdiff[i] > 0.03):
    st_fr.append(np.int(v1[i]))
    end_fr.append(np.int(v2[i]) )
print(st_fr)
print(end_fr)
len(st_fr)
st_fr = np.array(st_fr)
end_fr = np.array(end_fr)
featex = np.stack((st_fr, end_fr), axis = 1)
print(featex)
np.savetxt('/content/grad_cut_gt_pred.txt', featex, fmt='%s', delimiter=' ')
print(len(featex))

#EXTRACTING GROUNDTRUTH FROM XML FILE FOR ABRUPT CUT
import numpy as np
import xml.etree.ElementTree as ET
mytree = ET.parse('/content/ref_nad57.xml')
myroot = mytree.getroot()
print(int(myroot.attrib.get('totalFNum')))
totfno = int(myroot.attrib.get('totalFNum'))
prefno = []
postno = []
pref = []
postf = []
classi1 = []
cl = []
for x in myroot.findall('trans'):
  it1 = x.attrib.get('preFNum')

  it2 = x.attrib.get('postFNum')
  item =x.attrib.get('type')
  #print(item)
  prefno.append(int(it1))
  postno.append(int(it2))
  pref.append("frame%d.jpg" % int(it1))
  postf.append("frame%d.jpg" % int(it2))
  classi1.append(item)
  pre11 = []
  post11 = []
for i in range(len(classi1)):
  if (classi1[i]=='DIS' or classi1[i]=='FOI'):
    pre11.append(prefno[i])
    post11.append(postno[i])

#cl1 = np.array(cl)
pre1 = np.array(pre11)
post1 = np.array(post11)

feat1 = np.stack((pre1, post1), axis = 1)
print(feat1)
print(len(feat1))

np.savetxt('/content/gr_cut_gt.txt', feat1, fmt='%s', delimiter=' ')

#pre = 0
#RECALL CALCULATION
mat = np.zeros((len(feat1), len(featex)))
for i in range(0, len(featex)):
  a = featex[i][0]
  b = featex[i][1]
  for j in range(0, len(feat1)):
      r = range(feat1[j][0], feat1[j][1] + 1) 
      if((a in r) or (b in r)):
        mat[j][i] = 1

print(mat)
#print(mat[0][8])
print(len(mat))
 
print(mat.sum(axis = 1))
pre_mat = mat.sum(axis = 1)
print(pre_mat)
print(pre_mat.shape)
print(pre_mat[0])
print(np.sum(pre_mat))

pre = 0
for i in range(0, len(pre_mat)):
  a = pre_mat[i]
  #print(a)
  if(a!= 0.0):
    pre = pre + 1

print("recall")
print(pre/len(pre_mat))


Streaming output truncated to the last 5000 lines.
8
no of -1
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
1
1
1
1
no of -1
2
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
1
1
1
1
no of -1
2
1
1
1
1
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
no of +1
10
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
1
1
no of -1
2
1
1
no of +1
2
1
1
no of -1
2
1
1
no of +1
2
no of +1
3
no of +1
4
1
1
1
1
no of +1
2
no of +1
3
no of +1
4
no of +1
5
no of +1
6
no of +1
7
no of +1
8
no of +1
9
1
1
no of -1
2
no of -1
3
no of -1
4
no of -1
5
no of -1
6
no of -1
7
no of -1
8
no of -1
9
no of -1
10
no of -1
11
no of -1
12
no of -1
13
no of -1
14
no of -1
15
no of -1
16
no of -1
17
no of -1
18
no of -1
19
no of -1
20
1
1
1
1
no of -1
2
no of -1
3
no of -1
4
1
1
no of +1
2
no of +1
3
1
1
no of -1
2
no of -1
3
no of -1
4
1
1
no of +1
2
1
1
no of -1
2
no of -1
3
no of -1
4
no